In [ ]:
# Train torchvision with our dataset:
# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
# https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

from __future__ import division, print_function

import copy
import json
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

from money_counter import data


# Torch enable cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("=======================================================================")
print(" PyTorch version:     ", torch.__version__)
print(" Torchvision version: ", torchvision.__version__)
print(" Device:              ", device)
print("=======================================================================")


In [ ]:
ASSETS_DIR = '../assets'
DATASET_DIR = f'{ASSETS_DIR}/coins_dataset'
MODEL_STATE_DIR = f'../model_state'

Load the model

In [ ]:
import typing
from money_counter import data, models
from torch.utils.data import Subset

# Load the model
model, model_name = models.get_fasterrcnn_v2_pretrained()

# Load the latest weights
version_manager = models.VersionManager(MODEL_STATE_DIR)


version_manager.load_model(model_name, model, mode="best")


Load the data

In [ ]:
data_loader_train, data_loader_test = data.get_data_loaders(DATASET_DIR)


Draw the first 10 images with the boxes

In [ ]:
from typing import List
from money_counter.models import PredictedTarget
from money_counter.validation_utils import render_image_and_boxes
from money_counter import constants as const
from money_counter.engine import apply_nms

model = model.to(device)
model.eval()

label_map = {label: i for i, label in enumerate(const.CLASSES)}

with torch.no_grad():
    for _ in range(5):
        # Get some images from disk
        images, targets = next(iter(data_loader_test))
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Compute predictions
        predicted_targets = typing.cast(List[PredictedTarget], model(images))

        # apply nms
        apply_nms(predicted_targets, nms_threshold=0.5)

        # Render the image and the predictions
        for image, target, predicted in zip(images, targets, predicted_targets):
            # Move back to CPU
            image = image.cpu()
            target = {k: v.cpu() for k, v in target.items()}
            predicted = {k: v.cpu() for k, v in predicted.items()}

            subset_test = typing.cast(Subset, data_loader_test.dataset)
            data_set_test = typing.cast(data.CoinsDataset, subset_test.dataset)

            inverted_label_map = {v: k for k, v in label_map.items()}

            render_image_and_boxes(
                image,  inverted_label_map, None, predicted, 0.5)
